In [1]:
import os
import sys

import json

import numpy as np
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report

import scarches as sca
from scarches.dataset.trvae.data_handling import remove_sparsity
import matplotlib.pyplot as plt

from lightning.pytorch.callbacks import ModelCheckpoint

import torch

from dotenv import load_dotenv

from lightning.pytorch.loggers import WandbLogger
import wandb

import session_info
import warnings
from pyprojroot.here import here

#plt.style.use(['science','nature','no-latex'])
dpi_fig_save = 300
sc.set_figure_params(dpi=100, dpi_save=dpi_fig_save, vector_friendly=True)

# Setting some parameters
warnings.filterwarnings("ignore")

from sklearn.model_selection import StratifiedKFold

overwriteData = True
overwriteFigures = True

# Set random seed
random_seed = 42

import warnings
warnings.filterwarnings('ignore')

import scvi
scvi.settings.dl_num_workers = 0
scvi.settings.seed = random_seed

#torch.set_float32_matmul_precision('high')
#torch.set_printoptions(precision=3, sci_mode=False, edgeitems=7)
# torch.multiprocessing.set_sharing_strategy('file_system')

%load_ext autoreload
%autoreload 2

/home/dmaspero/miniconda3/envs/scarches/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
 captum (see https://github.com/pytorch/captum).


INFO: Seed set to 42


INFO:lightning.fabric.utilities.seed:Seed set to 42


In [2]:
assert load_dotenv()

In [3]:
workingDir = here('04_visualizing_final_embedding_space/SCGT00_CentralizedDataset/')
workingDir

PosixPath('/scratch_isilon/groups/singlecell/shared/projects/Inflammation-PBMCs-Atlas/04_visualizing_final_embedding_space/SCGT00_CentralizedDataset/02_scANVI_integration_with_annotation')

In [4]:
class CustomWandbLogger(WandbLogger):
    @property
    def save_dir(self):
        """Gets the save directory.

        Returns:
            The path to the save directory.

        """
        return self.experiment.dir

In [5]:
scvi.__version__

'1.1.2'

# Loading data


In [6]:
# Load the h5ad file
adata = sc.read_h5ad(here(f"{workingDir}/results/scVI_model_pretreined/adata.h5ad"))#, 
                     #backed='r+', chunk_size=50000)


#### Preparing scANVI training

In [7]:
scvi_model = sca.models.SCVI.load(here(f"{workingDir}/results/scVI_model_pretreined_batches/"), adata=adata) 

INFO     File                                                                                                      
         /scratch_isilon/groups/singlecell/shared/projects/Inflammation-PBMCs-Atlas/04_visualizing_final_embedding_space/SCGT00_CentralizedDataset/02_scANVI_integration_wit
         h_annotation/results/scVI_model_pretreined_batches/model.pt already downloaded                            


Outdated cuSPARSE installation found.
Version JAX was built against: 12200
Minimum supported: 12100
Installed version: 12002
The local installation version must be no lower than 12100. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


### Fine tuning with scANVI

**Parameters**

In [8]:
scANVI_trainer_kwargs = dict(
    n_samples_per_label = None,
    check_val_every_n_epoch = None,
    train_size = 0.8,
    validation_size = 0.2,
    shuffle_set_split = True,
    checkpointing_monitor = 'elbo_validation',
    early_stopping_monitor = 'reconstruction_loss_validation',
    early_stopping_patience = 2,
    early_stopping_min_delta=0.1,
    early_stopping = True,
    max_epochs = 1000,
)
# https://docs.scvi-tools.org/en/stable/api/reference/scvi.train.TrainingPlan.html#scvi.train.TrainingPlan
plan_kwargs = dict(
    lr = 5e-5,
    #reduce_lr_on_plateau = True
)
datasplitter_kwargs = dict(pin_memory=False)
scanvi_parameter_dict = scANVI_trainer_kwargs | plan_kwargs | datasplitter_kwargs

In [9]:
run_name = f"MAINobj_scANVI_fineTuning_lowLR_batches"
run_name

'MAINobj_scANVI_fineTuning_lowLR_batches'

In [10]:
scanvi_model = sca.models.SCANVI.from_scvi_model(scvi_model, unlabeled_category = "unknown")
scanvi_model

ScanVI Model with the following params: 
unlabeled_category: unknown, n_hidden: 256, n_latent: 30, n_layers: 4, dropout_rate: 0.1, dispersion: gene, 
gene_likelihood: nb
Training status: Not Trained
Model's adata is minified?: False

In [11]:
logger = CustomWandbLogger(name = run_name, project='inflammation_atlas_R1_scANVI', config = scanvi_parameter_dict)

In [12]:
scanvi_model.train(logger=logger, plan_kwargs = plan_kwargs, datasplitter_kwargs=datasplitter_kwargs, **scANVI_trainer_kwargs)

INFO     Training for 1000 epochs.                                                                                 


INFO: GPU available: True (cuda), used: True


INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True


INFO: TPU available: False, using: 0 TPU cores


INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores


...

Epoch 18/1000:   2%|▋                                       | 17/1000 [58:01<59:58:54, 219.67s/it, v_num=qitt, train_loss_step=2.08e+3, train_loss_epoch=1.99e+3]

Epoch 18/1000:   2%|▋                                     | 18/1000 [1:01:27<58:46:44, 215.48s/it, v_num=qitt, train_loss_step=2.08e+3, train_loss_epoch=1.99e+3]

Epoch 18/1000:   2%|▋                                     | 18/1000 [1:01:27<58:46:44, 215.48s/it, v_num=qitt, train_loss_step=1.88e+3, train_loss_epoch=1.99e+3]

Epoch 18/1000:   2%|▋                                     | 18/1000 [1:01:27<55:53:05, 204.87s/it, v_num=qitt, train_loss_step=1.88e+3, train_loss_epoch=1.99e+3]


Monitored metric reconstruction_loss_validation did not improve in the last 2 records. Best score: 1964.297. Signaling Trainer to stop.


In [13]:
wandb.finish()

wandb:                                                                                


wandb: 
wandb: Run history:
wandb:                     elbo_train ▄██▇▆▅▄▄▃▃▃▃▂▂▂▂▁▁
wandb:                elbo_validation ██▇▆▅▅▄▄▃▃▃▃▂▂▂▂▁▁
wandb:                          epoch ▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇████
wandb:                kl_global_train ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:           kl_global_validation ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:                 kl_local_train ▁██▇▆▅▅▄▄▃▃▃▃▂▂▂▁▁
wandb:            kl_local_validation ▇█▇▆▅▅▄▄▄▃▃▃▃▂▂▂▁▁
wandb:      reconstruction_loss_train █▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
wandb: reconstruction_loss_validation █▅▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁
wandb:                 train_accuracy ▁▄▅▅▆▆▆▆▇▇▇▇▇▇████
wandb:        train_calibration_error █▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁
wandb:      train_classification_loss █▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁
wandb:                 train_f1_score ▁▄▅▅▆▆▆▆▇▇▇▇▇▇████
wandb:               train_loss_epoch █▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
wandb:                train_loss_step ▇▆▅▂▆▄▇▆▆▇▃▄▂▃▇▂█▅▁█▇▆▅▄▃▆▄▄█▇▃▇▅▆▄▅▃▄▆▄
wandb:            trainer/global_step ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇

wandb: 🚀 View run MAINobj_scANVI_fineTuning_lowLR_batches at: https://wandb.ai/inflammation/inflammation_atlas_R1_scANVI/runs/725hqitt/workspace
wandb: Synced 6 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)


wandb: Find logs at: ./wandb/run-20241022_195249-725hqitt/logs


In [14]:
if overwriteData:
    scanvi_model.save(here(f"{workingDir}/results/scANVI_model_fineTuned_lowLR_batches/"), 
                      overwrite = True, 
                      save_anndata = True)

In [15]:
scanvi_emb = scanvi_model.get_latent_representation(adata=adata)

In [16]:
np.savez_compressed(file = str(here(f"{workingDir}/results/scANVI_model_fineTuned_lowLR_batches/scANVI_embedding.npz")), arr=scanvi_emb)